In [1]:
from pathlib import Path
from copy import deepcopy

import numpy as np
from pymatgen.core import Element

from chic import Structure

In [2]:
# set path to cif file.
eg_path = Path('UiO-66(Zr)')

# load structure.
struct = Structure.from_cif(eg_path / 'UiO-66_cdc-H2O.cif')

# compute neighbour list.
struct.get_neighbours_crystalnn(cores=8)

# find atomic clusters.
struct.find_atomic_clusters()

# find metal clusters.
struct.find_metal_clusters()

/Users/tcnicholas/miniconda3/envs/ace2023/lib/python3.9/site-packages/chic/cif.py:107: UserWarning: Unable to match CIF atoms to Pymatgen atoms.
  warnings.warn('Unable to match CIF atoms to Pymatgen atoms.')


get_neighbours_crystalnn() took 17.15 seconds to execute.
find_atomic_clusters() took 0.02 seconds to execute.


ValueError: No image found for site index 243, and no visited neighbor with an image to use as a reference.

In [2]:
240 // 120

2

In [ ]:
struct.atomic_clusters[('a', 1)].visualise()

No module named 'phonopy'


In [ ]:
def add_hydroxyl_proton(
    struct, 
    cluster, 
    r0: float,
    tetrahedron_orientation=None
):
    """
    Add a hydrogen atom to a meant-to-be-hydroxyl group.

    Arguments:
        struct: pymatgen Structure object.
        cluster: the cluster to add the hydrogen atom to.
        r0: the distance to the hydrogen atom (Angstrom).
        tetrahedron_orientation: Optional. The relative orientation of the tetrahedron.
    """

    # Get the centroid of the cluster and the coordinates of the oxygens.
    centroid = cluster.get_centroid()
    oxygen_coords = cluster.get_cart_coords_by_species('O')

    # Initialise the hydrogen coordinates.
    hydrogen_coords = np.zeros((4, 3))

    if tetrahedron_orientation is None:

        # determine the orientation of the tetrahedron.
        tetrahedron_orientation = []
        tetrahedral_oxygen_indices = [1, 3, 2, 6]
        for i, index in enumerate(tetrahedral_oxygen_indices):
            direction = oxygen_coords[index] - centroid
            tetrahedron_orientation.append(direction.copy())
            direction /= np.linalg.norm(direction)
            hydrogen_coords[i] = oxygen_coords[index] + r0 * direction
            
    else:
        
        # Find the 4 oxygens closest to the provided tetrahedron orientation
        for i in range(4):
            distances = np.linalg.norm(
                oxygen_coords - (centroid + tetrahedron_orientation[i]), 
                axis=1
            )
            min_index = np.argmin(distances)
            hydrogen_coords[i] = oxygen_coords[min_index] + r0 * tetrahedron_orientation[i] / np.linalg.norm(tetrahedron_orientation[i])
    
    # Add hydrogen atoms to the structure
    for coords in hydrogen_coords:
        struct.append(Element('H'), coords, coords_are_cartesian=True)

    return np.array(tetrahedron_orientation)

In [3]:
# make a deepcopy of the structure.
struct_copy = deepcopy(struct)
a_clusters = [struct.atomic_clusters[('a', i)] for i in range(1,5)]

# add hydrogen atoms to the structure.
tetrahedron_orientation = None
for i, cluster in enumerate(a_clusters):
    tetrahedron_orientation = add_hydroxyl_proton(
        struct_copy, 
        cluster, 
        1.0, 
        tetrahedron_orientation
    )

# save the structure.
struct_copy.to(str(eg_path / 'UiO-66_cdc-H2O-H.cif'), 'cif');

NameError: name 'add_hydroxyl_proton' is not defined

In [7]:
# now we have a structure with hydroxyl groups, we can model the structure.
# first we need to sort the structure. then write it.
struct_copy.sort()
struct_copy.to(str(eg_path / 'POSCAR'), 'poscar');